In [1]:
from itertools import repeat
import collections.abc

In [2]:
def _ntuple(n):
    def parse(x):
        if isinstance(x, collections.abc.Iterable):
            return x
        return tuple(repeat(x, n))
    return parse


In [9]:
to_2tuple = _ntuple(2)

In [2]:
import torch.nn as nn
import torch

In [10]:
m1 = nn.MaxPool2d(3, 2, 1)
m2 = nn.AvgPool2d(3, 2, 1)
input = torch.randn(20, 16, 25, 25)
output1 = m1(input)
output2 = m2(input)

In [11]:
output1.shape, output2.shape

(torch.Size([20, 16, 13, 13]), torch.Size([20, 16, 13, 13]))

In [22]:
class eca_layer(nn.Module):
    """Constructs a ECA module.
    Args:
        channel: Number of channels of the input feature map
        k_size: Adaptive selection of kernel size
    """
    def __init__(self, channel, k_size=3, stride=1, for_v3=False):
        super(eca_layer, self).__init__()
        self.avg_pool = nn.AdaptiveAvgPool2d(1)
        self.conv = nn.Conv1d(1, 1, kernel_size=k_size, padding=(k_size - 1) // 2, bias=False) 
        self.for_v3 = for_v3
        self.sigmoid = nn.Sigmoid()


    def forward(self, x):
        # x: input features with shape [b, c, h, w]
        b, c, h, w = x.size()
        print(b, c, h, w)
        # feature descriptor on the global spatial information
        y = self.avg_pool(x)
        print(y.shape)
        # Two different branches of ECA module
        print(y.squeeze(-1).transpose(-1, -2).shape)
        y = self.conv(y.squeeze(-1).transpose(-1, -2)).transpose(-1, -2).unsqueeze(-1)
        print(y.shape)
        # Multi-scale information fusion
        y = self.sigmoid(y)

        return x * y.expand_as(x)


In [23]:
input = torch.randn(20, 16, 25, 25)

In [24]:
model1 = eca_layer(16, 5, 1)

In [25]:
output1 = model1(input)

20 16 25 25
torch.Size([20, 16, 1, 1])
torch.Size([20, 1, 16])
torch.Size([20, 16, 1, 1])


In [21]:
output1.shape

torch.Size([20, 16, 25, 25])